# Multiple Variable Wavelet Preprocessing

In [1]:
%load_ext autoreload
%autoreload 2
p = print

from os.path import join

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from scipy import signal
import pywt

from crypr.util import get_project_path
from crypr.build import make_features, data_to_supervised

In [2]:
SYM = 'BTC'
TARGET = 'close'
Tx = 72
Ty = 1
TEST_SIZE = 0.05

data_path = join(get_project_path(), 'data', 'raw', SYM + '.csv')
data = pd.read_csv(data_path, index_col=0)
data.head()

,volumeto,volumefrom,open,high,close,low,time,timestamp
0,11679250.09,1420.13,8228.41,8231.84,8196.22,8188.29,1532836800,2018-07-29 06:00:00
1,8812752.64,1072.57,8196.44,8226.30,8211.36,8190.38,1532840400,2018-07-29 07:00:00
2,7966303.56,969.86,8211.37,8214.82,8200.12,8199.24,1532844000,2018-07-29 08:00:00
3,10236329.48,1246.57,8200.12,8216.15,8178.36,8175.14,1532847600,2018-07-29 09:00:00
4,13369556.62,1637.23,8178.36,8179.49,8153.85,8134.48,1532851200,2018-07-29 10:00:00


In [3]:
"""
Get percent change feature and target data.
"""
df = make_features(input_df=data, target_col='close')
X, y = data_to_supervised(input_df=df, target_ix=-1, Tx=Tx, Ty=Ty)
p(X.shape, y.shape)
X.head()

(5926, 72) (5926, 1)


,var1(t-72),var1(t-71),var1(t-70),var1(t-69),var1(t-68),var1(t-67),var1(t-66),var1(t-65),var1(t-64),var1(t-63),...,var1(t-10),var1(t-9),var1(t-8),var1(t-7),var1(t-6),var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1)
72,0.184719,-0.136884,-0.265362,-0.299693,0.130491,-0.175761,0.167850,0.563094,-0.177836,0.233308,...,0.089263,-0.749809,0.205035,-0.126087,0.364724,-0.757954,-1.407772,-0.132389,-0.112852,-0.102119
73,-0.136884,-0.265362,-0.299693,0.130491,-0.175761,0.167850,0.563094,-0.177836,0.233308,0.183582,...,-0.749809,0.205035,-0.126087,0.364724,-0.757954,-1.407772,-0.132389,-0.112852,-0.102119,-0.072524
74,-0.265362,-0.299693,0.130491,-0.175761,0.167850,0.563094,-0.177836,0.233308,0.183582,-0.589959,...,0.205035,-0.126087,0.364724,-0.757954,-1.407772,-0.132389,-0.112852,-0.102119,-0.072524,0.541471
75,-0.299693,0.130491,-0.175761,0.167850,0.563094,-0.177836,0.233308,0.183582,-0.589959,0.112213,...,-0.126087,0.364724,-0.757954,-1.407772,-0.132389,-0.112852,-0.102119,-0.072524,0.541471,-0.174064
76,0.130491,-0.175761,0.167850,0.563094,-0.177836,0.233308,0.183582,-0.589959,0.112213,0.244688,...,0.364724,-0.757954,-1.407772,-0.132389,-0.112852,-0.102119,-0.072524,0.541471,-0.174064,0.958425


In [4]:
"""
Confirm data reshape and target/feature creation was done correctly.
"""
y_values_except_last = np.squeeze(y.iloc[:-1].values)
t_minus_1_x_values_except_first = X.iloc[1:,-1].values

y_values_except_last.all() == t_minus_1_x_values_except_first.all()

True

In [5]:
"""
For comparing different transformations
"""
sample_ix = 1000

In [6]:
"""
Reshape the data into 3d array if multiple variables.
"""
X = X.values.reshape((X.shape[0], -1, Tx))
p(X.shape)

(5926, 1, 72)


In [7]:
"""
Apply the wave transformation to the feature data.
"""
wt_type = 'DWT_HAAR'
p('Applying {} transform ...'.format(wt_type))

if wt_type == 'RICKER':
    wt_transform_fun = lambda x: signal.cwt(x, wavelet=signal.ricker, widths=widths)
elif wt_type == 'HAAR':
    wt_transform_fun = lambda x: Haar(x).getpower()
elif wt_type == 'DWT_HAAR':
    wt_transform_fun = lambda x: np.stack(pywt.dwt(x, 'haar'))
else:
    raise NotImplementedError
    
X_wt = np.apply_along_axis(func1d=wt_transform_fun, axis=-1, arr=X)

X_wt.shape

Applying DWT_HAAR transform ...


(5926, 1, 2, 36)

In [8]:
"""
Condense wavelet features if multiple features analyzed.
"""
X_wt = X_wt.reshape((X_wt.shape[0], X_wt.shape[1]*X_wt.shape[2], X_wt.shape[-1]))
N = X_wt.shape[-2:]
X_wt.shape, N

((5926, 2, 36), (2, 36))

In [9]:
"""
Reshape the data so Tx is the 2nd dimension.
"""
X_wt_rs = X_wt.swapaxes(-1,-2)
p(X_wt_rs.shape)

(5926, 36, 2)


In [10]:
"""
Train Test Split.
"""
X_train, X_test, y_train, y_test = train_test_split(X_wt_rs, y, test_size=TEST_SIZE, shuffle=False)

In [11]:
"""
Save data.
"""
output_dir = join(get_project_path(), 'data', 'processed')

np.save(arr=X_train, allow_pickle=True, 
        file=join(output_dir, '.X_train_{}_{}_{}x{}'.format(SYM, wt_type, Tx, N)))
np.save(arr=X_test, allow_pickle=True, 
        file=join(output_dir, 'X_test_{}_{}_{}x{}'.format(SYM, wt_type, Tx, N)))
np.save(arr=y_train, allow_pickle=True, 
        file=join(output_dir, 'y_train_{}_{}_{}x{}'.format(SYM, wt_type, Tx, N)))
np.save(arr=y_test, allow_pickle=True, 
        file=join(output_dir, 'y_test_{}_{}_{}x{}'.format(SYM, wt_type, Tx, N)))